In [1]:
import cv2
import numpy as np
import os
import json
from flask import Flask,request,render_template,session,Response
from flask_cors import CORS

#Our lib
import Utilities as ut
import Database_user as db_user
import Database_emotion as ss
import Analysis as magic
import Alerts 
import Outer_api as api

In [2]:
app = Flask(__name__)
app.secret_key = 'your_secret_key'

In [3]:
#flutter
@app.route('/uploadx', methods=['POST'])
def uploadx():
    framesArray = []
    user = request.files['username']
    for i in range(126):
        file = request.files[f'frame{i}']
        frame_bytes = file.read()
        np_frame = np.frombuffer(frame_bytes, np.uint8)
        frame = cv2.imdecode(np_frame, cv2.IMREAD_COLOR)
        framesArray.append(frame)
    ut.sendToModel(framesArray , user)
    return Response(response='Frames received', status=200)

In [4]:
#website
@app.route('/', methods=['GET', 'POST'])
def login():
    if request.method == 'POST':
        username = request.form['username']
        password = request.form['password']
        if db_user.authPass(username,password):
            session['value'] = username
            return render_template('x.html' , username = username)
        else:
            error_message = 'Invalid credentials. Please try again.'
        return render_template('x2.html', error_message=error_message)
    return render_template('x2.html')

@app.route('/upload', methods=['POST'])
def upload():
    video_file = request.files['video']
    video_path = os.path.join('D:/Downloads/videos', video_file.filename) 
    video_file.save(video_path)
    frames =ut.getFrames(video_path)
    username = request.form.get('username')
    x = ut.sendToModel(frames , username)
    return x

In [5]:
#client
@app.route('/chatgpt/<username>/<int:TimeInsec>', methods=['GET'])
def chatgpt(username,TimeInsec):
    TimeInsec = TimeInsec//5
    dic =magic.analysis(username ,TimeInsec )
    response = api.chatGpt(dic['overallEmotion'])
    return Response(response=response, status=200)

@app.route('/image/<username>/<int:TimeInsec>', methods=['GET'])
def image(username,TimeInsec):
    TimeInsec = TimeInsec//5
    dic =magic.analysis(username ,TimeInsec )
    responsez = api.image()
    return Response(response=responsez, status=200)

@app.route('/spotify/<username>/<int:TimeInsec>', methods=['GET'])
def spotify(username,TimeInsec):
    TimeInsec = TimeInsec//5
#     dic =magic.analysis(username ,TimeInsec )
    responsee = api.spotify()
    return Response(response=responsee, status=200)

In [6]:
#supervisor

@app.route('/getAlerts/<PatientName>/<int:TimeInsec>', methods=['GET'])
def getAlerts(PatientName,TimeInsec):
    Alerts.send(PatientName,TimeInsec)
    response = Alerts.getAlerts(PatientName)
    response = json.dumps(response)
    return Response(response=response, status=200)

@app.route('/getReports/<username>/<int:TimeInsec>', methods=['GET'])
def getReports(username , TimeInsec):
    response = magic.analysis(username ,TimeInsec//5 )['report']
    response = json.dumps(response)
    return Response(response=response, status=200)

@app.route('/login/<username>/<password>', methods=['GET'])
def loginz(username,password):
        if db_user.authPass(username,password):
            return Response(response=db_user.getType(username), status=200)
        else:
            return Response(response = "no" , status = 200)
        
@app.route('/signup/<username>/<password>/<type>/<superVisorName>', methods=['GET'])
def signup(username,password,type,superVisorName):
    if not db_user.checkSuperVisor(superVisorName):
        return Response(response = "false" , status = 200)
    if  db_user.checkIfUserExist(username):
        return Response(response = "false" , status = 200)
    db_user.signup(username , password , type , superVisorName)
    return Response(response = "done" , status = 200)

@app.route('/getUser/<SupervisorName>', methods=['GET'])
def getUser(SupervisorName):
    response = db_user.getUsers(SupervisorName)
    response = json.dumps(response)
    return Response(response = response , status = 200)

@app.route('/AdvancedReport/<patientName>/<int:TimeInsec>', methods=['GET'])
def MainReportPage(patientName,TimeInsec):        
    response = magic.analysis(patientName ,TimeInsec//5 )
    response = ut.handlePlots(response)
    response = json.dumps(response)
    return Response(response = response , status = 200)

In [ ]:
if __name__ == "__main__":
    #host='0.0.0.0', port=80
    app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [12/Jun/2023 17:46:49] "GET /login/mohamed+emad/123456 HTTP/1.0" 200 -
127.0.0.1 - - [12/Jun/2023 17:46:56] "GET /login/barooudy/123456 HTTP/1.0" 200 -
127.0.0.1 - - [12/Jun/2023 17:47:06] "GET /login/baroudy/123456 HTTP/1.0" 200 -
127.0.0.1 - - [12/Jun/2023 17:47:29] "GET /login/baroudy/123456 HTTP/1.0" 200 -


1/1 [==============================] - 0s 266ms/step
['neg']


In [ ]:
# for i in range(60):
#     ss.store('baroudy', "pos")
# for i in range(60):
#     ss.store('baroudy', "neg")